In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import preprocessingWithMissingvalues

carDf= preprocessingWithMissingvalues.preprocess('../ann/cardata.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
Month                   15420 non-null object
WeekOfMonth             15420 non-null int64
DayOfWeek               15420 non-null object
Make                    15420 non-null object
AccidentArea            15420 non-null object
DayOfWeekClaimed        15420 non-null object
MonthClaimed            15420 non-null object
WeekOfMonthClaimed      15420 non-null int64
Sex                     15420 non-null object
MaritalStatus           15420 non-null object
Age                     15420 non-null int64
Fault                   15420 non-null object
PolicyType              15420 non-null object
VehicleCategory         15420 non-null object
VehiclePrice            15420 non-null object
FraudFound              15420 non-null object
PolicyNumber            15420 non-null int64
RepNumber               15420 non-null int64
Deductible              15420 non-null int64
DriverRating           

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;
carDf[0:2]

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,AddressChange-Claim_under 6 months,NumberOfCars_1 vehicle,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,RepNo,Deductible,DriverRating,DaysDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0.733333,0.00,0.0,0.071795
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.933333,0.25,1.0,0.069231


In [4]:
#apply the PCA
from sklearn.decomposition import PCA

carDf = StandardScaler().fit_transform(carDf)


pca = PCA(.95)
pca_result = pca.fit_transform(carDf)
#pca_df = pd.DataFrame(columns = ['pca1','pca2','pca3','pca4'])

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [5]:
print(type(pca_result))

<class 'numpy.ndarray'>


In [6]:
pca_result= pd.DataFrame(pca_result)
pca_result.head()

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,8.385616,3.854620,-2.064169,2.316098,0.587232,-1.553042,-1.340943,3.049831,-1.770466,10.359703,...,22.472592,-23.907883,26.256987,60.084584,3.724137,15.336528,9.814129,1.489515,-3.001330,-5.697575
1,2.847836,-2.088748,-0.386049,2.777993,3.492078,4.313151,-1.597910,0.637190,-1.133189,-1.285239,...,0.061326,0.260817,-0.659223,-0.221485,1.909207,1.759892,1.099109,0.143473,-1.505178,-0.724217
2,-0.451507,-0.428698,0.565207,3.293118,0.234022,-0.427259,-1.256547,0.691969,1.267305,0.333467,...,-0.585429,-0.146577,-0.465165,-0.251005,1.444187,1.557063,0.951375,0.010909,-1.622217,-1.128883
3,-1.889307,0.077026,0.863538,-0.406910,4.688306,3.972506,0.874295,-2.097643,-2.236658,0.098089,...,0.323089,-0.379188,-0.440375,-0.059298,0.559230,0.849083,-0.295101,0.335771,-0.242074,1.055900
4,3.852191,-2.335400,-0.636814,1.696282,2.128785,-1.504042,-0.879829,1.887913,1.541650,2.341173,...,-0.906605,-0.069863,-0.727519,-0.373084,1.469164,2.177938,0.343096,-0.191737,-1.295324,-0.945496


In [7]:
carDateNormalized= pca_result

In [8]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1

In [9]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 69)
(923, 69)


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,8.385616,3.854620,-2.064169,2.316098,0.587232,-1.553042,-1.340943,3.049831,-1.770466,10.359703,...,22.472592,-23.907883,26.256987,60.084584,3.724137,15.336528,9.814129,1.489515,-3.001330,-5.697575
1,2.847836,-2.088748,-0.386049,2.777993,3.492078,4.313151,-1.597910,0.637190,-1.133189,-1.285239,...,0.061326,0.260817,-0.659223,-0.221485,1.909207,1.759892,1.099109,0.143473,-1.505178,-0.724217
2,-0.451507,-0.428698,0.565207,3.293118,0.234022,-0.427259,-1.256547,0.691969,1.267305,0.333467,...,-0.585429,-0.146577,-0.465165,-0.251005,1.444187,1.557063,0.951375,0.010909,-1.622217,-1.128883
3,-1.889307,0.077026,0.863538,-0.406910,4.688306,3.972506,0.874295,-2.097643,-2.236658,0.098089,...,0.323089,-0.379188,-0.440375,-0.059298,0.559230,0.849083,-0.295101,0.335771,-0.242074,1.055900
4,3.852191,-2.335400,-0.636814,1.696282,2.128785,-1.504042,-0.879829,1.887913,1.541650,2.341173,...,-0.906605,-0.069863,-0.727519,-0.373084,1.469164,2.177938,0.343096,-0.191737,-1.295324,-0.945496


In [10]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [11]:
nonFraudulent.head()
fraudulentLable.head()

,lable
28,1.0
52,1.0
53,1.0
94,1.0
96,1.0


In [12]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulent, fraudulentLable,random_state=3,test_size=0.25)
print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)

nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 69)
fraudxtrain: <class 'pandas.core.frame.DataFrame'>
(231, 69)


In [13]:
#convert into numpy
nonFraudX_train= nonFraudX_train.values
nonFraudX_test= nonFraudX_test.values
nonFraudY_train= nonFraudY_train.values
nonFraudY_test= nonFraudY_test.values

fraudX_train= fraudX_train.values
fraudX_test= fraudX_test.values
fraudY_train= fraudY_train.values
fraudY_test= fraudY_test.values


In [14]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [15]:
#make the  nonfraudulent dataset for train and test loader

myNonFraudulentTrainDataset= myDataset(nonFraudX_train,nonFraudY_train)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make the fraudulent dataset for train and test loader
myFraudulentTrainDataset= myDataset(fraudX_train,fraudY_train)
myFraudulentTestDataset= myDataset(fraudX_test,fraudY_test)

In [16]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

fraudulentTrainLoader= torch.utils.data.DataLoader(myFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
fraudulentTestLoader= torch.utils.data.DataLoader(myFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)



In [17]:
#Network architecture for the base autoencoders
class autoencoder1(nn.Module):
    def __init__(self):
        super(autoencoder1,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(69,59),
            nn.ReLU(True),
            nn.Linear(59,49),
            nn.ReLU(True),
            nn.Linear(49,39),
            nn.ReLU(True),
            nn.Linear(39,29),
            nn.ReLU(True),
            nn.Linear(29,19),nn.ReLU(True),nn.Linear(19,10))
        
        self.decoder1= nn.Sequential(
            nn.Linear(10,19),
            nn.ReLU(True),
            nn.Linear(19,29),
            nn.ReLU(True),
            nn.Linear(29,39),nn.ReLU(True),nn.Linear(39,49),
            nn.ReLU(True),nn.Linear(49,59),
            nn.ReLU(True),nn.Linear(59,69),nn.Tanh())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [18]:
model1= autoencoder1()
criterion1= nn.MSELoss()
optimizer1= torch.optim.SGD(model1.parameters(), lr=0.001,momentum=0.9)

In [19]:
#train the model
for epoch in range(200):
    running_loss= 0.0
    for i, (feature1,lable1) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs1= torch.tensor(feature1)
        lables1= torch.tensor(lable1)
        lables1= lables1.type(torch.LongTensor)
        
        # =====================forward====================
        output1 = model1(inputs1)
        loss1 = criterion1(output1,inputs1)
        
         # ===================backward====================
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        
        # =======print the statistics
        running_loss += loss1.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss /nonFraudX_train.shape[0]))
      #running_loss = 0.0

        

[1] loss: 1.316
[2] loss: 1.316
[3] loss: 1.316
[4] loss: 1.316
[5] loss: 1.316
[6] loss: 1.316
[7] loss: 1.316
[8] loss: 1.316
[9] loss: 1.316
[10] loss: 1.316
[11] loss: 1.316
[12] loss: 1.316
[13] loss: 1.316
[14] loss: 1.316
[15] loss: 1.316
[16] loss: 1.316
[17] loss: 1.316
[18] loss: 1.316
[19] loss: 1.316
[20] loss: 1.316
[21] loss: 1.316
[22] loss: 1.316
[23] loss: 1.316
[24] loss: 1.316
[25] loss: 1.316
[26] loss: 1.316
[27] loss: 1.316
[28] loss: 1.316
[29] loss: 1.316
[30] loss: 1.316
[31] loss: 1.316
[32] loss: 1.316
[33] loss: 1.316
[34] loss: 1.316
[35] loss: 1.316
[36] loss: 1.316
[37] loss: 1.316
[38] loss: 1.316
[39] loss: 1.316
[40] loss: 1.316
[41] loss: 1.316
[42] loss: 1.316
[43] loss: 1.316
[44] loss: 1.316
[45] loss: 1.316
[46] loss: 1.316
[47] loss: 1.316
[48] loss: 1.316
[49] loss: 1.316
[50] loss: 1.316
[51] loss: 1.316
[52] loss: 1.316
[53] loss: 1.316
[54] loss: 1.316
[55] loss: 1.316
[56] loss: 1.316
[57] loss: 1.316
[58] loss: 1.316
[59] loss: 1.316
[60] l

In [20]:
 #test the model
def modelTest1(Loader):
    lossList1= []
    trueLable1= []
    for i,(feature1,lable1) in enumerate(Loader):
        inputs1= torch.tensor(feature1)
        output1= model1(inputs1)
        loss1= criterion1(output1,inputs1)
        trueLable1.append(lable1)
        lossList1.append(loss1.item())
        lossValue1= loss1.item()
        f= open("./modle1Loss.txt",'a')
        f.write(str(lossValue1) + '\n')
        '''
        if(i!=10):
            print("input:",inputs1)
            print("output:",output1)
            print("loss1:",loss1)
            print("lable:",lable1)
        else:
            assert(False)
        '''
    f.close()
    return lossList1,trueLable1
        

In [32]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder2= nn.Sequential(
            nn.Linear(69,45),
            nn.LeakyReLU(True),
            nn.Linear(45,25),
            nn.LeakyReLU(True),
            nn.Linear(25,10))
       
        self.decoder2= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,45),
            nn.LeakyReLU(True),
            nn.Linear(45,69),nn.Tanh())
    
    def forward(self,x):
        x= self.encoder2(x)
        x= self.decoder2(x)
        return x;

In [33]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [34]:
#train the model2
for epoch in range(75):
    running_loss2= 0.0
    for i, (feature2,lable2) in enumerate(fraudulentTestLoader):
        
        #gets the inputs
        inputs2= torch.tensor(feature2)
        lables2= torch.tensor(lable2)
        lables2= lables2.type(torch.LongTensor)
       
        # =====================forward====================
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        
        # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudX_train.shape[0]))
      #running_loss = 0.0

[1] loss: 0.472
[2] loss: 0.464
[3] loss: 0.457
[4] loss: 0.451
[5] loss: 0.446
[6] loss: 0.442
[7] loss: 0.438
[8] loss: 0.433
[9] loss: 0.428
[10] loss: 0.422
[11] loss: 0.416
[12] loss: 0.410
[13] loss: 0.405
[14] loss: 0.400
[15] loss: 0.396
[16] loss: 0.391
[17] loss: 0.388
[18] loss: 0.385
[19] loss: 0.382
[20] loss: 0.380
[21] loss: 0.377
[22] loss: 0.375
[23] loss: 0.373
[24] loss: 0.371
[25] loss: 0.370
[26] loss: 0.368
[27] loss: 0.366
[28] loss: 0.365
[29] loss: 0.363
[30] loss: 0.362
[31] loss: 0.361
[32] loss: 0.360
[33] loss: 0.358
[34] loss: 0.357
[35] loss: 0.356
[36] loss: 0.355
[37] loss: 0.354
[38] loss: 0.352
[39] loss: 0.351
[40] loss: 0.350
[41] loss: 0.349
[42] loss: 0.348
[43] loss: 0.347
[44] loss: 0.346
[45] loss: 0.345
[46] loss: 0.344
[47] loss: 0.344
[48] loss: 0.343
[49] loss: 0.342
[50] loss: 0.341
[51] loss: 0.340
[52] loss: 0.339
[53] loss: 0.339
[54] loss: 0.338
[55] loss: 0.337
[56] loss: 0.336
[57] loss: 0.335
[58] loss: 0.335
[59] loss: 0.334
[60] l

In [35]:
def modelTest2(Loader2):
    lossList2= []
    trueLable1= []
    for i,(feature2,lable2) in enumerate(Loader2):
        inputs2= torch.tensor(feature2)
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        trueLable1.append(lable2)
        lossList2.append(loss2.item()) #put the loss
        lossValue2= str(loss2.item())
        f= open("./modle2Loss.txt",'a')
        f.write(lossValue2+'\n')
    f.close()
        
    return lossList2,trueLable1
        

In [36]:
#test the model1
lossList1,trueLable1= modelTest1(nonFraudulentTestLoader)

#test the model2
lossList2,trueLable1= modelTest2(nonFraudulentTestLoader)

In [37]:
predictedLable1=[]
for k in range(len(lossList1)):
    if(lossList1[k]<lossList2[k]):
        predictedLable1.append(0)
    else:
        predictedLable1.append(1)

In [38]:
#test model1 with another type of data
lossList3,trueLable2= modelTest1(fraudulentTestLoader)


In [39]:
#test model2 with another type of data
lossList4,trueLable2= modelTest2(fraudulentTestLoader)


In [40]:
predictedLable2=[]
for k in range(len(lossList3)):
    if(lossList3[k]<lossList4[k]):
        predictedLable2.append(0)
    else:
        predictedLable2.append(1)

In [41]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)



Result 
Accuracy: 52.2697795071336
Sensitivity: 60.17316017316018
Specificity: 51.76600441501103



TP: 139

FN: 92

FP: 1748

TN: 1876


In [54]:
def testModel1(loader):
    lossList= []
    predLable= []
    for i,(feature,lable) in enumerate(loader):
        inputs= torch.tensor(feature)
        output= model1(inputs)
        loss= criterion2(output,inputs)
        if(i!=10):
            print("loss:",loss.item())
        else:
            i= i
            
        if(loss > 0.9):
            predLable.append(1)
        else:
            predLable.append(0)
    return predLable
        

In [58]:
predLable= testModel1(fraudulentTestLoader)

loss: 2.988356113433838
loss: 0.38591378927230835
loss: 1.100298285484314
loss: 1.6101622581481934
loss: 0.6091102957725525
loss: 0.6388607621192932
loss: 0.9478121995925903
loss: 0.5564133524894714
loss: 0.4326455295085907
loss: 0.3188246488571167
loss: 0.39330604672431946
loss: 0.5118290185928345
loss: 0.8938324451446533
loss: 1.1824462413787842
loss: 0.3533158302307129
loss: 0.3919377624988556
loss: 0.5036945343017578
loss: 0.36006638407707214
loss: 0.9403480291366577
loss: 0.7970864772796631
loss: 0.4299071133136749
loss: 0.4689094126224518
loss: 0.635746955871582
loss: 1.1725484132766724
loss: 0.32587313652038574
loss: 0.5336700677871704
loss: 2.6759581565856934
loss: 0.40714001655578613
loss: 0.8285308480262756
loss: 0.3975133001804352
loss: 1.1992175579071045
loss: 1.3588886260986328
loss: 0.8351196646690369
loss: 0.3204759359359741
loss: 0.2898620367050171
loss: 1.4170210361480713
loss: 0.3854910135269165
loss: 1.1189616918563843
loss: 1.046439290046692
loss: 0.6354686617851257

class1= predLable.count(1)
class0= predLable.count(0)

In [57]:
print(class1)
print(class0)

1043
2581
